In [22]:
import boto3
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

### data

In [23]:
client = boto3.client('s3')

In [24]:
s3_prefix = 's3://'
bucket = 'sisyphus-general-bucket'
primary_folder = 'AthenaInsights'
data_folder = 'latest_data'

In [25]:
response = client.list_objects_v2(
    Bucket=bucket,
    Prefix=f'{primary_folder}/{data_folder}/reduced_autocorelation/')

In [26]:
files = []
all_symbols = set()

In [27]:
paths = []
for content in response.get('Contents', []):
    # print(f"{s3_prefix}{bucket}/{content['Key']}")
    paths.append(f"{s3_prefix}{bucket}/{content['Key']}")

In [28]:
def calculate_ma(df, ema=True, sma=True, calc_windows=[], prices=[]):
    # Function to apply moving averages
    def apply_moving_averages(group):
        for window in tqdm(calc_windows):
            for price in prices:
                if ema:
                    group[f'{price}_ema_{window}m'] = group[price].ewm(span=window, adjust=False).mean()
                if sma:
                    group[f'{price}_sma_{window}m'] = group[price].rolling(window=window).mean()
        return group

    # Apply function by group
    return df.groupby('symbol').apply(apply_moving_averages)

In [29]:
col_sets = [['open', 'high', 'low', 'close', 'volume', 'symbol', 'time_delta'], 
            ['open_diff', 'high_diff', 'low_diff', 'close_diff', 'volume_diff', 'symbol', 'time_delta']]

In [30]:
for path in paths:
    name = path.split('/')[-1].split('.')[0].split('_')[-1]
    if 'min' in name:
        window = int(name.replace('min', ''))
        period = 'min'
    elif 'D' in name:
        window = int(name.replace('D', ''))
        period = 'D'
    if period=='D':
        if window < 100:
            calc_windows = [2, 3, 5, 8, 10, 12, 15, 20, 26, 50, 100, 200]
        else: 
            calc_windows = [5, 8, 10, 12, 20, 26, 50]
    else:
        if window <= 30:
            calc_windows = [2, 3, 5, 8, 12, 10, 20, 26]
        else:
            calc_windows = [5, 8, 10, 12, 20, 26, 50, 100, 200]
    print(path)
    df = pd.read_parquet(path)
    df1 = calculate_ma(df[col_sets[0]], calc_windows=calc_windows, prices=col_sets[0][:-2]).droplevel(0)
    df1.to_parquet(path.replace('reduced_autocorelation', 'feature_prep').replace('.parquet', '_base.parquet'))
    print(f"written to {path.replace('reduced_autocorelation', 'feature_prep').replace('.parquet', '_base.parquet')}")    
    del df1
    df2 = calculate_ma(df[col_sets[1]], calc_windows=calc_windows, prices=col_sets[1][:-2]).droplevel(0)
    df2.to_parquet(path.replace('reduced_autocorelation', 'feature_prep').replace('.parquet', '_diff.parquet'))
    print(f"written to {path.replace('reduced_autocorelation', 'feature_prep').replace('.parquet', '_diff.parquet')}")    
    del df2

s3://sisyphus-general-bucket/AthenaInsights/latest_data/reduced_autocorelation/stock_bars_100D.parquet


100%|██████████| 7/7 [00:00<00:00, 212.47it/s]


written to s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_100D_base.parquet


100%|██████████| 7/7 [00:00<00:00, 376.42it/s]


written to s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_100D_diff.parquet
s3://sisyphus-general-bucket/AthenaInsights/latest_data/reduced_autocorelation/stock_bars_10D.parquet


...

written to s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_5min_diff.parquet
s3://sisyphus-general-bucket/AthenaInsights/latest_data/reduced_autocorelation/stock_bars_60min.parquet


100%|██████████| 9/9 [00:00<00:00, 228.73it/s]


written to s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_60min_base.parquet


100%|██████████| 9/9 [00:00<00:00, 287.48it/s]


written to s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_60min_diff.parquet
